# Build a wine quality prediction model using K-NN
## Determine efficacy of scaling

In [5]:
# Imports....the usual suspects
import postgresql
import pandas
import sklearn
import matplotlib
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

## 1. Download the Wine Quality Dataset from the UCI
http://archive.ics.uci.edu/ml/datasets/Wine+Quality
Look for the 'Data Folder' link near top

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=1)
df = pandas.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', delimiter=';')
df.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


In [16]:
# Apply our encoding magic!
for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
        
x_df = df.drop(columns=['quality'])
y_df = df['quality']


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


## 2. Split the Red Wine Dataset into training and testing

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df)
        


## 3. Build a K-NN Regression Model to predict the continuous quality variable in the Red Wine Dataset.  Predict for your test set (you're estimating quality column)


In [19]:
# 1 Neighbor, no preprocessing

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.58


## 4. What is the mean of the differences (absolute value) between the Y actuals (quality column) and your rounded (to the nearest integer) predictions?

In [37]:
y_actual = pandas.DataFrame(y_test)
y_actual['prediction'] = clf.predict(x_test)
y_actual['pred_rounded'] = y_actual['prediction'].round().astype(int)
y_actual['error'] = y_actual['pred_rounded'] - y_actual['quality']
y_actual.head()


,quality,prediction,pred_rounded,error
55,5,6,6,1
1075,7,6,6,-1
1411,6,6,6,0
1487,5,6,6,1
566,6,6,6,0


In [34]:
y_actual['error'].abs().mean()

0.4775

## 5. Repeat steps 2-5 after scaling the data.  Try at least two types of scaling.  

In [20]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [21]:
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.65


In [23]:
#RobustScaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [24]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.56


## 6. Did Scaling help?

In [ ]:
#Yeah, in this CASE only the 'StandardScaler' increases Accuracy but 'Robust Scalar' did not works.

## 7. The quality column holds integers.  Therefore, it would also work to build a classifier for this column (discrete variable).  Do you think it would be better or worse to use a classifier?  Why?

In [ ]:
#Its better only to use Classifier because it assigns Labels to a class of Input data, here its not good because its an integer and we are not able to identify the wine.